In [39]:
import numpy as np
import cv2
import os



In [126]:
# test_data = cv2.imread("./data/train(region)/masks/2012_01_02__22_59_43_84__SDO_AIA_AIA_193.png",
#                        cv2.IMREAD_GRAYSCALE)
# res, _ = cv2.findContours(test_data, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

# np.shape(res)
# np.shape(create_contours(test_data))
# create_contours(test_data)

In [127]:
def create_contours(mask):
    
    width = mask.shape[1]
    height = mask.shape[0]
    # ret, thresh = cv2.threshold(mask, 127, 255, 0)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    coords = []
    for i, object in enumerate(contours):
        coords.append([])
        for point in object:
            new_x = (int(point[0][0])) / (width)
            new_y = (int(point[0][1])) / (height)
            coords[i].append((new_x, new_y))
    return coords

In [128]:
def convert_to_yolo(folder_path):
    masks_path = os.path.join(folder_path, 'masks')
    labels_path = os.path.join(folder_path, 'labels')

    png_files = [file for file in os.listdir(masks_path) if file.endswith('.png')]

    for png_file in png_files:
        input_path = os.path.join(masks_path, png_file)

        image_cv2 = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

        coords_of_objects = create_contours(image_cv2)
    
        txt_file = os.path.splitext(png_file)[0] + '.txt'
        output_path = os.path.join(labels_path, txt_file)

        with open(output_path, 'w') as label_file:
            for i, object_coords in enumerate(coords_of_objects):
                coordinates_str = ' '.join(map(str, np.reshape(object_coords, -1)))
                string_to_write = f"0 {coordinates_str}"
                if i < (len(coords_of_objects) - 1):
                    string_to_write += "\n"
                label_file.write(string_to_write)

In [4]:
!python3 -m pip install --upgrade Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.2.0
    Uninstalling Pillow-9.2.0:
      Successfully uninstalled Pillow-9.2.0


In [3]:
!pip3 install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 27.3 MB/s eta 0:00:0000:0100:01


In [26]:
import cv2
from PIL import Image

def display_label(label_name):
    image = cv2.imread("./data/train(region)/images/"+label_name+".png")

    with open("./data/train(region)/labels/" + label_name + ".txt", 'r') as f:
        label = f.read()

    label_elements = label.strip().split(' ')

    coords = [(int(float(label_elements[i])*256), int(float(label_elements[i+1])*256)) for i in range(1, len(label_elements), 2)]
   
    for coord in coords:
        cv2.circle(image, coord, radius=0, color=(0, 255, 0), thickness=-1)
   
    cv2.imwrite("./test.png", image)


In [15]:
import cv2
   
def convert_to_grayscale(folder_path):
    images_path = os.path.join(folder_path, 'images')
    grayscale_images_path = os.path.join(folder_path, 'images-gray')

    png_files = [file for file in os.listdir(images_path) if file.endswith('.png')]
    
    for img_path in png_files:
        image = cv2.imread(images_path + "/" + img_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        cv2.imwrite(grayscale_images_path + "/" + img_path, gray_image)


In [35]:
from pathlib import Path

def remove_invalid_labels_with_images(data_path):
    images_path = os.path.join(data_path, 'images')
    labels_path = os.path.join(data_path, 'labels')

    txt_files = [file for file in os.listdir(labels_path) if file.endswith('.txt')]
    
    for txt_file in txt_files:
        with open(labels_path +"/"+ txt_file, 'r') as f:
            label = f.read()
            png_file = os.path.splitext(txt_file)[0] + '.png'
            if(len(label.split()) < 5):
                print("removing... " + png_file)
                Path(images_path+"/" + png_file).unlink()
                print("removing... " + txt_file)
                Path(labels_path+"/" + txt_file).unlink()
        
            

In [37]:
# remove_invalid_labels_with_images("./data/train(region+grayscale)")

In [17]:
convert_to_grayscale("./data/val(region+grayscale)")

In [38]:
display_label("2012_01_31__22_59_43_84__SDO_AIA_AIA_193")

In [129]:
convert_to_yolo("./data/train(region)")
convert_to_yolo("./data/val(region)")